In [5]:
import pandas
import numpy
import pygsheets
import datetime
import pytz
import re

In [6]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

In [7]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A')
spreadsheet = gc.open_by_key('1xedJW5v5DMat4-pdC_qYzBH-L2PLsr6-N-daYD8dzgI') # June 4 for Wenying

terms_df_orig = spreadsheet.worksheet('title', 'Terminals').get_as_df(start='A3')

# drop any empty/imcomplete rows (those without a wiki page)
terms_df_orig = terms_df_orig.loc[terms_df_orig.Wiki != '']

#get other relevant sheets
owners_df_orig = spreadsheet.worksheet('title', 'Terminal operators/owners (1/3)').get_as_df(start='A2')
owners_df_orig.set_index('ComboID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [8]:
terms_df_orig.replace('--', numpy.nan, inplace=True)

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_82517/2923329017.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  terms_df_orig.replace('--', numpy.nan, inplace=True)


In [11]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')
#region_df_agt = region_df_orig.loc[region_df_orig['AsiaGasTracker']=='Yes']

region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
#region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
#region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

In [50]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = terms_df_orig.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_country = terms_df_orig.groupby(['Region','Country'])['Country'].count().index
multiindex_region_subregion

MultiIndex([(  'Africa',                 'Northern Africa'),
            (  'Africa',              'Sub-Saharan Africa'),
            ('Americas', 'Latin America and the Caribbean'),
            ('Americas',                'Northern America'),
            (    'Asia',                    'Central Asia'),
            (    'Asia',                    'Eastern Asia'),
            (    'Asia',              'South-eastern Asia'),
            (    'Asia',                   'Southern Asia'),
            (    'Asia',                    'Western Asia'),
            (  'Europe',                  'Eastern Europe'),
            (  'Europe',                 'Northern Europe'),
            (  'Europe',                 'Southern Europe'),
            (  'Europe',                  'Western Europe'),
            ( 'Oceania',       'Australia and New Zealand'),
            ( 'Oceania',                       'Melanesia')],
           names=['Region', 'SubRegion'])

In [51]:
multiindex_region_country_tuples = list(zip(multiindex_region_country.get_level_values(0),multiindex_region_country.get_level_values(1)))

all_region_country_year_tuples = []
for item in multiindex_region_country_tuples:
    for year in year_list:
        all_region_country_year_tuples.append(tuple((item[0], item[1], year)))

### create specific dataframes for region, country_ratios, owners_df

In [52]:
owners_df_touse = owners_df_orig.loc[owners_df_orig['Country'].isin(region_df_touse['Country'].tolist())]
terms_df_touse = terms_df_orig.loc[terms_df_orig['Country'].isin(region_df_touse['Country'].tolist())]

# export

In [53]:
year_list = list(range(2015,2024))

In [54]:
# proposed
df1 = terms_df_touse.rename(columns={'ProposalYear':'Year'}).loc[(terms_df_touse.Status=='Proposed') & 
(terms_df_touse.FacilityType=='Export')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Proposed mtpa'})

In [55]:
# proposed
df2 = terms_df_touse.rename(columns={'ConstructionYear':'Year'}).loc[(terms_df_touse.Status=='Construction') & 
(terms_df_touse.FacilityType=='Export')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Under construction mtpa'})

In [56]:
# proposed
df3 = terms_df_touse.rename(columns={'StartYearEarliest':'Year'}).loc[(terms_df_touse.Status=='Operating') & 
(terms_df_touse.FacilityType=='Export')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Operating mtpa'})

In [57]:
# proposed
df4 = terms_df_touse.rename(columns={'ShelvedYear':'Year'}).loc[(terms_df_touse.Status=='Shelved') & 
(terms_df_touse.FacilityType=='Export')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Shelved mtpa'})

In [58]:
# proposed
df5 = terms_df_touse.rename(columns={'CancelledYear':'Year'}).loc[(terms_df_touse.Status=='Cancelled') & 
(terms_df_touse.FacilityType=='Export')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Cancelled mtpa'})

In [59]:
all_dfs = df1.join(df2).join(df3).join(df4).join(df5)

In [60]:
country_list = terms_df_touse.Country.unique().tolist()
region_list = terms_df_touse.Region.unique().tolist()

In [61]:
#new_multiindex = pandas.MultiIndex.from_product([region_list,country_list,year_list], names=['Country', 'Region', 'Year'])
new_multiindex = pandas.MultiIndex.from_tuples(all_region_country_year_tuples, names=['Country','Region','Year'])

In [62]:
all_dfs.reindex(new_multiindex).reset_index().replace(
    numpy.nan,'').to_excel('GGIT-yearly-stats-LNG-export-terminals.xlsx', index=False)

# import

In [63]:
# proposed
df1 = terms_df_touse.rename(columns={'ProposalYear':'Year'}).loc[(terms_df_touse.Status=='Proposed') & 
(terms_df_touse.FacilityType=='Import')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Proposed mtpa'})

In [64]:
# proposed
df2 = terms_df_touse.rename(columns={'ConstructionYear':'Year'}).loc[(terms_df_touse.Status=='Construction') & 
(terms_df_touse.FacilityType=='Import')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Under construction mtpa'})

In [65]:
# proposed
df3 = terms_df_touse.rename(columns={'StartYearEarliest':'Year'}).loc[(terms_df_touse.Status=='Operating') & 
(terms_df_touse.FacilityType=='Import')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Operating mtpa'})

In [66]:
# proposed
df4 = terms_df_touse.rename(columns={'ShelvedYear':'Year'}).loc[(terms_df_touse.Status=='Shelved') & 
(terms_df_touse.FacilityType=='Import')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Shelved mtpa'})

In [67]:
# proposed
df5 = terms_df_touse.rename(columns={'CancelledYear':'Year'}).loc[(terms_df_touse.Status=='Cancelled') & 
(terms_df_touse.FacilityType=='Import')].groupby(
    ['Region','Country','Year']
        )[['CapacityInMtpa']].sum().rename(columns={'CapacityInMtpa':'Cancelled mtpa'})

In [68]:
all_dfs = df1.join(df2).join(df3).join(df4).join(df5)

In [69]:
all_dfs.reindex(new_multiindex).reset_index().replace(
    numpy.nan,'').to_excel('GGIT-yearly-stats-LNG-import-terminals.xlsx', index=False)